## The Copy-Paste Database Problem {#the-copy-paste-database-problem}

> 📖 Read the full article: [Stop Writing SQL for AI Agents: Build Direct Database Access with FastMCP](https://codecut.ai/fastmcp-sql-bridge-database-access/)



AI has revolutionized how we write code, making complex database queries accessible to anyone who can describe what they need. But the workflow becomes frustrating when you're trapped in this repetitive cycle: ask your AI for SQL, copy the query, paste it into your database tool, run it, copy the results, and paste them back to your AI for analysis.

Here's what this workflow looks like in practice:

```bash
# 1. Ask AI for SQL
"Can you write a query to find high-value customers?"

# 2. Copy AI's response
SELECT customer_id, total_spent FROM customers WHERE total_spent > 1000

# 3. Paste into database tool, run query, copy results
# 4. Paste results back to AI for analysis
```

This manual approach creates several challenges:

- Switching between AI and database tools breaks your analytical flow
- Copying and pasting introduces transcription errors
- AI can't explore data independently or learn from previous queries

What if your AI could connect directly to your database, run queries autonomously, and provide insights without you ever copying and pasting SQL? That's exactly what you'll build in this article using MCP.

## What is Model Context Protocol (MCP)? {#what-is-model-context-protocol-mcp}

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/) is a standard that allows AI models to connect directly to external systems like databases, APIs, and file systems. Instead of being limited to generating code or text, MCP enables AI models to take actions and access real data.

Think of MCP as a bridge between your AI assistant and your database. When you ask AI to "find high-value customers," the assistant discovers your database tools, calls the appropriate functions, and provides actionable insights—all without you leaving the conversation.

![Diagram illustrating MCP client-server communication with AI assistant requesting database tools and receiving responses](https://codecut.ai/wp-content/uploads/2025/07/diagram-export-7-10-2025-3_12_42-PM.png)

MCP works through two key components:

- **Tools**: Functions that AI models can call to perform actions and modify system state (like executing SQL queries, inserting data, exporting files)
- **Resources**: Data sources that AI models can access for information only (like table schemas, sample data, or documentation)

![Diagram showing MCP tools vs resources: tools perform actions like execute_query while resources provide read-only access to schemas and data](https://codecut.ai/wp-content/uploads/2025/07/diagram-export-7-10-2025-3_12_21-PM.png)

This direct connection eliminates the copy-paste cycle and enables truly autonomous data analysis.

## What is FastMCP? {#what-is-fastmcp}

[FastMCP](https://github.com/jlowin/fastmcp) is a Python framework that makes building MCP servers incredibly simple. While you could build MCP servers from scratch, FastMCP provides decorators and utilities that reduce the complexity to just a few lines of code.

In this article, you'll build a database assistant that:

- Connects to SQLite databases and executes queries automatically
- Exports results to CSV files with full schema discovery
- Provides sample data and calculates database statistics

This complete solution transforms any AI assistant into a powerful database analytics tool. For large-scale analytics workloads, consider our [DuckDB deep dive](https://codecut.ai/deep-dive-into-duckdb-data-scientists/) which offers superior performance for analytical queries.




## Installation and Database Setup {#installation-and-database-setup}

### Installation {#installation}

To install fastmcp, type the following command:

```bash
pip install fastmcp
```

Other dependencies you'll need for this article are:

```bash
pip install sqlalchemy pandas
```

[SQLAlchemy](https://docs.sqlalchemy.org/) is the Python SQL toolkit and Object-Relational Mapping (ORM) library we'll use for database operations.

## Database Setup: Sample E-commerce Data {#database-setup-sample-e-commerce-data}

Before building FastMCP tools, let's create a realistic e-commerce database that we'll use throughout all examples. The complete database setup code is available in [`setup_database.py`](https://github.com/khuyentran1401/fastmcp_examples/blob/main/setup_database.py).

```python
from setup_database import create_sample_database

# Create the database we'll use throughout the article
db_path = create_sample_database("ecommerce.db")
```

## Building Your First Database Tools {#building-your-first-database-tools}

In this section, you'll create FastMCP tools that:

- Connect to databases and manage connections
- Discover tables and database structure
- Execute queries with proper transaction handling
- Export results to CSV files

### Connect to the Database {#connect-to-the-database}

Start with creating a tool called `connect_db` to connect to the SQLite database. To create a tool, you need to:

- Initialize FastMCP server
- Write a Python function with your tool logic
- Add the `@mcp.tool` decorator

In [ ]:
from fastmcp import FastMCP
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# Global database connection
db_engine = None
db_session_factory = None

# Initialize FastMCP server
mcp = FastMCP("Database Analytics Assistant")

@mcp.tool
def connect_db(database_path: str) -> dict:
    """Connect to an SQLite database file"""
    global db_engine, db_session_factory

    # Create new engine and session factory
    db_engine = create_engine(f"sqlite:///{database_path}")
    db_session_factory = sessionmaker(bind=db_engine)

    return {"success": True, "database_path": database_path}

FastMCP tools return JSON responses that AI assistants convert into user-friendly text. This structured approach ensures reliable data exchange between your database and AI systems. For type-safe AI responses, check out our [PydanticAI guide](https://codecut.ai/enforce-structured-outputs-from-llms-with-pydanticai/).

Here's what you'll see when calling the `connect_db` tool with the `ecommerce.db` path:

**JSON output:**

```json
{
    "success": true,
    "database_path": "ecommerce.db"
}
```

**AI-generated summary:**


```text
Connected to the database at ecommerce.db
```


### List Tables {#list-tables}

The `list_tables` tool uses SQLAlchemy's inspector to discover all tables in the connected database. This gives the AI a complete view of your database structure.


In [ ]:
@mcp.tool
def list_tables() -> dict:
    """List all tables in the connected database"""
    global db_engine
    from sqlalchemy import inspect

    inspector = inspect(db_engine)
    table_names = inspector.get_table_names()

    return {"success": True, "tables": table_names}

Here's the response you'll get when calling the `list_tables` tool:

**Human input:**

```text
Show me all tables in the ecommerce database
```

**AI converts this to tool call:** `list_tables()`


**JSON output:**

```json
{
    "success": true,
    "tables": ["users", "orders"]
}
```

**AI-generated summary:**

```text
The database has two tables: users and orders.
```


### Flexible Query Execution Tool {#flexible-query-execution-tool}

The `execute_query` tool executes only SELECT queries on the connected database for security. This read-only approach prevents accidental data modification or deletion while allowing powerful data analysis and exploration.

Start by creating a helper function to check if the query is safe.

In [ ]:
def _is_safe_query(sql: str) -> bool:
    """Check if a SQL query is safe to execute. Only SELECT queries are allowed."""
    sql_lower = sql.lower().strip()
    return sql_lower.startswith("select")

Then, create the `execute_query` tool.

In [ ]:
@mcp.tool
def execute_query(sql: str, ctx: Context = None) -> Dict[str, Any]:
    """Execute a SQL query on the connected database."""
    global db_session_factory

    # Check if query is safe before execution
    if not _is_safe_query(sql):
        return {
            "success": False,
            "error": "Potentially dangerous SQL operations are not allowed. Only SELECT queries are permitted."
        }

    with db_session_factory() as session:
        # Execute the SQL query
        result = session.execute(text(sql))
        rows = result.fetchall()
        return {"success": True, "results": [dict(row._mapping) for row in rows]}

Here's the response you'll get when calling the `execute_query` tool with the SQL query:


Human input:

```text
Show me statistics for the users table
```


AI converts this to:

```sql
SELECT
    COUNT(*) as total_users,
    AVG(age) as average_age,
    MIN(age) as youngest_age,
    MAX(age) as oldest_age,
    COUNT(CASE WHEN age < 30 THEN 1 END) as users_under_30,
    COUNT(CASE WHEN age >= 30 THEN 1 END) as users_30_and_over
FROM users
```

**JSON output:**

```json
{
    "success": true,
    "results": [
        {
            "total_users": 5,
            "average_age": 32.0,
            "youngest_age": 22,
            "oldest_age": 45,
            "users_under_30": 2,
            "users_30_and_over": 3
        }
    ]
}
```

**AI-generated summary:**

Users table statistics:

| Metric | Value |
|--------|-------|
| Total Users | 5 |
| Average Age | 32.0 |
| Youngest User | 22 |
| Oldest User | 45 |
| Users Under 30 | 2 |
| Users 30 and Over | 3 |


### Data Export Tool {#data-export-tool}

The `export_to_csv` tool executes a SQL query and exports the results to a CSV file. This is useful for creating reports or sharing data with others.

In [ ]:
import pandas as pd

@mcp.tool
def export_to_csv(sql: str, filename: str) -> dict:
    """Execute a SQL query and export results to CSV file"""
    global db_engine
    
    # Execute query and export to CSV using pandas
    df = pd.read_sql(sql, db_engine)
    df.to_csv(filename, index=False)
    
    return {"success": True, "filename": filename, "rows_exported": len(df)}

Here's the response you'll get when calling the `export_to_csv` tool:

**Human input:**

```text
Export product sales data to a CSV file called sales_report.csv
```

**AI converts this to:**

```sql
SELECT product_name, SUM(quantity) as total_sold, SUM(price * quantity) as total_revenue 
FROM orders 
GROUP BY product_name 
ORDER BY total_revenue DESC
```

**JSON output:**

```json
{
    "success": true,
    "filename": "sales_report.csv",
    "rows_exported": 7
}
```

**AI-generated summary:**

```text
Successfully exported 7 rows of product sales data to sales_report.csv
```


## Schema Discovery with Resources {#schema-discovery-with-resources}

Tools require complex security validation and can fail due to database errors or malformed queries. Resources eliminate these risks by providing read-only access to metadata like schemas and sample data without query execution overhead.

In this section, you'll create FastMCP resources that:

- Provide schema information for tables
- Provide sample data for tables
- Provide statistics for tables


### Table Schema Resource with URI Patterns {#table-schema-resource-with-uri-patterns}

The `get_table_schema` resource uses URI patterns to provide flexible access to table schemas. The AI can request schema information for any table by using the URI pattern `schema://tables/{table_name}`.

In [ ]:
from sqlalchemy import inspect

@mcp.resource("schema://tables/{table_name}")
def get_table_schema(table_name: str) -> dict:
    """Get column information for a specific table"""
    global db_engine

    # Get database inspector
    inspector = inspect(db_engine)

    # Get column information
    columns = inspector.get_columns(table_name)

    # Build column info list
    column_info = []
    for col in columns:
        column_info.append({
            "name": col["name"],
            "type": str(col["type"]),
            "nullable": col["nullable"],
        })

    return {"table_name": table_name, "columns": column_info}

Here's the response you'll get when calling the `get_table_schema` resource:

**Human input:**

```text
Show me the schema for the users table
```

**AI accesses the resource using URI:** `schema://tables/users`

**JSON output:**

```json
{
    "table_name": "users",
    "columns": [
        {
            "name": "id",
            "type": "INTEGER",
            "nullable": false
        },
        {
            "name": "name",
            "type": "TEXT",
            "nullable": false
        },
        {
            "name": "email",
            "type": "TEXT",
            "nullable": false
        },
        {
            "name": "age",
            "type": "INTEGER",
            "nullable": true
        },
        {
            "name": "created_at",
            "type": "TIMESTAMP",
            "nullable": true
        }
    ]
}
```

**AI-generated summary:**

The `users` table schema:

| Column | Type | Required |
|--------|------|----------|
| id | INTEGER | Yes |
| name | TEXT | Yes |
| email | TEXT | Yes |
| age | INTEGER | No |
| created_at | TIMESTAMP | No |


### Sample Data Resource {#sample-data-resource}

The `get_table_data` resource provides sample data with pagination support. This helps the AI understand the actual data structure and content without overwhelming it with large datasets.

In [ ]:
@mcp.resource("data://tables/{table_name}")
def get_table_data(table_name: str, limit: int = 10, offset: int = 0) -> dict:
    """Get sample rows from a specific table with pagination"""
    global db_session_factory

    with db_session_factory() as session:
        # Get sample data with pagination
        result = session.execute(
            text(f"SELECT * FROM {table_name} LIMIT :limit OFFSET :offset"),
            {"limit": limit, "offset": offset},
        )
        rows = result.fetchall()

        # Convert to dict
        data = [dict(row._mapping) for row in rows]

        return {"table_name": table_name, "sample_data": data, "rows_returned": len(data)}

Here's the response you'll get when calling the `get_table_data` resource:

**Human input:**

```text
Show me sample data from the users table
```

**AI accesses the resource using URI:** `data://tables/users`

**JSON output:**

```json
{
    "table_name": "users",
    "sample_data": [
        {
            "id": 1,
            "name": "Alice Johnson",
            "email": "alice@example.com",
            "age": 28,
            "created_at": "2023-01-15 10:30:00"
        },
        {
            "id": 2,
            "name": "Bob Smith",
            "email": "bob@example.com",
            "age": 35,
            "created_at": "2023-02-20 14:15:00"
        },
        {
            "id": 3,
            "name": "Charlie Brown",
            "email": "charlie@example.com",
            "age": 22,
            "created_at": "2023-03-10 09:45:00"
        }
    ],
    "rows_returned": 3
}
```

**AI-generated summary:**

Sample data from the `users` table:

| User ID | Name | Email | Age | Created |
|---------|------|-------|-----|---------|
| 1 | Alice Johnson | alice@example.com | 28 | 2023-01-15 |
| 2 | Bob Smith | bob@example.com | 35 | 2023-02-20 |
| 3 | Charlie Brown | charlie@example.com | 22 | 2023-03-10 |


### Table Statistics Resource {#table-statistics-resource}

The `get_table_stats` resource provides comprehensive statistics for a specific table. This helps the AI understand the size and composition of the table.

In [ ]:
@mcp.resource("stats://tables/{table_name}")
def get_table_stats(table_name: str) -> dict:
    """Get comprehensive statistics for a specific table"""
    global db_engine, db_session_factory

    with db_session_factory() as session:
        # Get basic table statistics
        total_rows = session.execute(
            text(f"SELECT COUNT(*) FROM {table_name}")
        ).scalar()

    # Get column information
    inspector = inspect(db_engine)
    columns = inspector.get_columns(table_name)

    return {
        "table_name": table_name,
        "total_rows": total_rows,
        "column_count": len(columns),
    }

Here's the response you'll get when calling the `get_table_stats` resource:

**Human input:**

```text
Show me statistics for the users table
```

**AI accesses the resource using URI:** `stats://tables/users`

**JSON output:**

```json
{
    "table_name": "users",
    "total_rows": 5,
    "column_count": 5
}
```

**AI-generated summary:**

The `users` table contains **5 rows** and has **5 columns** (id, name, email, age, created_at).


## Connect to MCP Clients {#connect-to-mcp-clients}

Now that you've built all the tools and resources, let's deploy your FastMCP database server and connect it to an MCP client.

An MCP client is any application that can communicate with MCP servers to access tools and resources, such as Claude Code, Claude Desktop, Cursor, Zed, Continue, and other MCP-compatible development tools.

### Create the Server File {#create-the-server-file}

First, combine all the code from this article into a single file called `database_mcp_server.py`. You can find the complete implementation in the [example repository](https://github.com/khuyentran1401/database-analytics-mcp.git).

### Install Dependencies {#install-dependencies}

Install [UV](https://github.com/astral-sh/uv) (the fast Python package manager) if you haven't already:

```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
```

Use `uv` (the fast Python package manager) to install all dependencies:

```bash
uv sync
```

## Connect with MCP Clients {#connect-with-mcp-clients}

This FastMCP server works with any MCP-compatible client. Here's how to connect it to Claude Code and other MCP clients.

### Connect to Claude Code {#connect-to-claude-code}

Add your server to Claude Code's MCP configuration:

```bash
claude mcp add database-analytics -- uv run database_mcp_server.py
```

Verify the server is registered:

```bash
claude mcp list
```

### Connect to Claude Desktop {#connect-to-claude-desktop}

Add this configuration to your Claude Desktop config file:

```json
{
  "mcpServers": {
    "database-analytics": {
      "command": "uv",
      "args": ["run", "database_mcp_server.py"],
      "cwd": "/path/to/your/project"
    }
  }
}
```

### Connect to other MCP clients {#connect-to-other-mcp-clients}

Popular MCP-compatible clients include Cursor, Zed, Continue, and Codeium. Each client has its own configuration format but uses the same server command: `uv run database_mcp_server.py`. For client-specific setup guides, visit the [Model Context Protocol documentation](https://modelcontextprotocol.io/clients).


## Test the Server {#test-the-server}

Once your FastMCP server is running and connected to an MCP client, you can test all the database functionality through natural language commands. Here are practical examples to verify everything works:

Connect to the database:

```text
Connect to my SQLite database at ./ecommerce.db
```

Output:

```text
✅ Connected to the database at ./ecommerce.db
```

Explore the schema:

```text
What tables are available in this database?
```

Output:
```text
The database has 2 tables: users and orders.
```

Examine the table structure:

```text
Show me the schema for the users table
```

Output:

```text
The users table has 5 columns: id (INTEGER, required), name (TEXT, required), email (TEXT, required), age (INTEGER, optional), and created_at (TIMESTAMP, optional).
```

Preview the data:

```text
Show me some sample data from the users table
```

Output:

```text
Here are 3 sample users:
- Alice Johnson (alice@example.com, age 28)
- Bob Smith (bob@example.com, age 35)  
- Charlie Brown (charlie@example.com, age 22)
```

Run analytics queries:

```text
Calculate total sales by product category
```

Output:

```text
Product sales summary:
- Laptop: $999.99 (1 sold)
- Tablet: $499.99 (1 sold)
- Monitor: $299.99 (1 sold)
- Headphones: $149.99 (1 sold)
```

Export the results:

```text
Export the query "SELECT product_name, SUM(quantity) as total_sold FROM orders GROUP BY product_name" to CSV file called sales_report.csv
```

Output:

```text
✅ Successfully exported 7 rows of product sales data to sales_report.csv
```

Get table statistics:

```text
Show me statistics for the users table
```

Output:

```text
Users table statistics: 5 total rows, 5 columns
```